In [1]:
from datetime import timedelta

from core.files_include import hist_file

threshold = 5
posThreshold = threshold
negThreshold = -threshold

from core.base_classes import BaseInt, BaseDate, BaseBuySell
from TradeUtil import *
#
# from base_classes import *
# from TradeUtil import *
header_lines = 6

# Symbol	Date	Quantity	Price	Amount	Account	Description

# Define Col Names
Symbol = 'Symbol'
Date = 'Date'
Quantity = 'Quantity'
Price = 'Price'
Amount = 'Amount'
Account = 'Account'
Description = 'Description'


@dataclass
class History(BaseTrade):
    Symbol: BaseTradeSymbol = None
    Date: BaseDate = None
    Quantity: BaseFloat = None
    Price: BaseTradePrice = None
    Amount: BaseTradePrice = None
    Account: BaseString = None
    Description: BaseBuySell = None

    def getPrice(self):
        return self.Price.getBase()
    def getQuantity(self):
        return self.Quantity.getBase()

    def isAnIdleSecurity(self):
        return self.Date.isOlderThan(45)

    def __str__(self):
        ps = str(self.Symbol) + "|" + str(self.Price)  + "|"+ str(self.Quantity)  + "|" + str(self.Description)
        return ps

    def noOfDaysSinceTrans(self):
        self.dayDiff = self.Date.getNoDaysFromToday() * -1
        return self.dayDiff

    def isOlderThan(self, days=30):
        return self.dayDiff > days

    def canBeSold(self):
        return self.isOlderThan()

    def isBuy(self):
        return self.getBuySell() == 'B'

    def isSell(self):
        return self.getBuySell() == 'S'

    def getBuySell(self):
        return self.Description.getBase()

    def matchesRefObj(self, bs):
        if bs == 'B':
            if self.isBuy():
                return self
        else:
            if self.isSell():
                return self
        return None

    @classmethod
    def from_dict(cls, data_dict):
        o =  cls(data_dict[Symbol],	data_dict['Date'], data_dict['Quantity'],
                   data_dict['Price'],	data_dict['Amount'],data_dict['Account'], data_dict['Description'])
        o.noOfDaysSinceTrans()
        return o
from account import AccountManager
@dataclass
class historySummary(BaseObject):
    historys : any #= None
    sym: BaseTradeSymbol #= None

    def _initbase(self):
        self.lastH = None
        self.lastAction = ""
        self.nextBuyPrice = 0
        self.nextSellPrice = 0
        self.lastPrice = 0.0
        self.lastQuantity = 0.0
        self.lastBuyRef = None
        self.lastSellRef = None
        self.lastActivityDate = None
        self.total_buy_qty  = 0
        self.total_sell_qty = 0
        self.total_buy_amt  = 0
        self.total_sell_amt = 0
        self.avg_buy_price  = 0
        self.avg_sell_price = 0
        self.total_gain_loss      = 0
        self.total_gain_loss_perc = 0
        self.avg_gain_loss_perc   = 0
        self.refObjs = []
        return
    def __post_init__(self):
        self._initbase()
        self.setForNextSym()
        return
    def isAnIdleSecurity(self):
        return self.history.isAnIdleSecurity()

    def getLastPrice(self):
        return self.lastPrice
    def getLastQuantity(self):
        return abs(self.lastQuantity)
    def getLastAction(self):
        return self.lastAction
    def getLastObj(self):
        return self.lastH
    def getNextBuyPrice(self):
        return self.nextBuyPrice
    def getNextSellPrice(self):
        return self.nextSellPrice
    def getRefObjs(self):
        return self.refObjs
    def getBuyRef(self):
        return self.refObjs[0]
    def getSellRef(self):
        return self.refObjs[1]

    def setForNextSym(self):
        self._setLastObj()
        self._setBuyRef()
        self._setSellRef()
        return

    def _setSummary(self):
        if self.lastQuantity > 0:
            self.total_buy_qty = self.total_buy_qty + self.lastQuantity
            self.total_buy_amt = self.total_buy_amt + self.lastH.getAmount()
        else:
            self.total_sell_qty = self.total_sell_qty + self.lastQuantity
            self.total_sell_amt = self.total_sell_amt + self.lastH.getAmount()
        return
    # def prepareSummary(self):
    #     self._setSummary()
    #     return self

    def _setLastObj(self):
        self.lastH = self._getRefObj()
        if self.lastH:
            self.lastPrice = self.lastH.getPrice()
            self.lastQuantity = self.lastH.getQuantity()
            self._setNextBuySellRefPrices()
            self._setLastAction()
            self._setLastActivityDate()
            # self.()_setSummary
        return

    def _setLastActivityDate(self):
        self.lastActivityDate = self.lastH.Date

    def _setNextBuySellRefPrices(self):
        self._setNextBuyPrice()
        self._setNextSellPrice()
        return
    def _setNextSellPrice(self):
        lp = self.getLastPrice()
        if self.lastAction.__eq__('B'):
            np = lp * (1 + .1)
        else:
            np = lp * (1 + .05)
        self.nextSellPrice = round(np, 2)
        return

    def _setNextBuyPrice(self):
        lp = self.getLastPrice()
        if self.lastAction.__eq__('S'):
            np = lp * (1 - .1)
        else:
            np = lp * (1 - .05)
        self.nextBuyPrice = round(np, 2)
        return

    def _setLastAction(self):
        if self.lastH.isBuy():
            self.lastAction = "B"
        else:
            self.lastAction = "S"
        return
    def hasHistoryPrices(self):
        if self.refObjs:
            if self.refObjs[0] and self.refObjs[1]:
                return True
        return False
    def _setBuyRef(self):
        hobj = self._getRefObj(bs="B")
        if not hobj:
            hobj = self._getRefObj(bs="S")
        self.refObjs.append(hobj)
        return
    def _setSellRef(self):
        hobj =  self._getRefObj(bs="S")
        if not hobj:
            hobj = self._getRefObj(bs="B")
        self.refObjs.append(hobj)
        return

    def _process_each_rec(self):
        recs = self.historys.findSymbol(self.sym)
        sobjs = recs.sort(key=lambda x: x.Date, reverse=True)
        for obj in sobjs:
            if isinstance(obj, History):
                self.hist_summ.append(obj)

        if isinstance(obj, History):
            self.historys.append(obj)
        return
    def _getRefObj(self, bs=None):
        recs = self.historys.findSymbol(self.sym, bs)
        if not recs:
            return None
        if recs.isEmpty():
            return None

        sobjs = recs.sort(key=lambda x: x.Date, reverse=True)
        if not bs:
            return sobjs[0]

        for obj in sobjs:
            if isinstance(obj, History):
                match = obj.matchesRefObj(bs)
                if match:
                    return match
        last_hobj = sobjs[0]
        return last_hobj
    def isAnIdleSecurity(self):
        if not self.lastActivityDate:
            return True
        if self.lastActivityDate.isOlderThan(45):
            return True
        return False
unittest = False
@dataclass
class Historys(BaseTrades):
    def __post_init__(self):
        super().__post_init__()
        sort_by = lambda x: x.Date
        self.presetTrades(sort_by=sort_by, reverese=True)
        self.cls = History
        self.uniqueCols = [	Symbol,	Amount,]
        # self.index_cols= ['Date']
        if unittest:
            hist_file = rootdir + "all_history_test.csv"
        self.readFile(self.cls, self.uniqueCols, header_lines=header_lines, datafile=hist_file)
        df = self.getDF()

        self.all_symbols = df[Symbol].unique()
        # self.prep_summ_df()
        self._histSumm = BaseDict()
        self.initHistorySummary()
        self.idle_trade_qry(45)
        self.postDFProcess()
        return

    def initHistorySummary(self):
        for sym in self.all_symbols:
            historys = self
            obj = historySummary(historys, sym)
            self._histSumm.append(sym, obj)
        return
    def getSummaryForSymbol(self, sym):
        if isinstance(sym, BaseObject):
            sym = sym.getBase()
        return self._histSumm.getValue(sym)

    def getSummaryObjForSymbol(self, sym):
        if isinstance(sym, BaseObject):
            sym = sym.getBase()
        return self._histSumm.getSummaryObj(sym)

    def postDFProcess(self):
        self.updateDateFormat()
        self.updateDFDescriptionToBS()
        self.history_since()
        return

    def updateDFDescriptionToBS(self):
        df = self.getDF()
        if not isinstance(df, DataFrame):
            return
        df[Description] = df[Description].apply(lambda x: "B" if x == "YOU BOUGHT" else ("S" if x == "YOU SOLD" else x))
        return

    def updateDateFormat(self):
        df = self.getDF()
        if not isinstance(df, DataFrame):
            return
        df[Date] = pd.to_datetime(df[Date], format='%m/%d/%Y')
        # df[Date] = df[Date].apply(lambda x: x.replace("/", "-"))
        return

    def getRecordsForSym(self, sym):
        df = self.getDF()
        return df.loc[df[Symbol] == sym]
    def getRecordForAcct(self, acct):
        df = self.getDF()
        return df.loc[df[Account] == acct]

    def getRecordForAcctAndSym(self, acct, sym, bs='B'):
        df = self.getDF()
        return df.loc[(df[Account] == acct) & (df[Symbol] == sym) & (df[Description] == bs)]

    def getLastBoughtForAcctAndSym(self, acct, sym):
        df = self.getRecordForAcctAndSym(acct, sym,  bs='B')
        if df.empty:
            return None
        df = df.sort_values(by=[Date], ascending=False)
        return df.iloc[0]

    def boughtInLastNDays(self, acct, sym, days=30):
        df = self.getRecordForAcctAndSym(acct, sym, bs='B')
        if df.empty:
            return False
        df = df.sort_values(by=[Date], ascending=False)
        lastBought = df.iloc[0]
        if lastBought.Date > pd.Timestamp.now() - pd.Timedelta(days=days):
            return True
        return False

    def getActListToSell(self, sym, p, no_of_days=30):
        # accts = self.boughtRecently(sym)
        accts = self.getHoldingAccounts()
        actSellList = BaseList()
        for act in accts.getBase():
            if not self.boughtInLastNDays(sym=sym, acct=act):
                if p.getCurrentObj(sym,  act):
                    actSellList.append(act)

        am = AccountManager()
        aidList = am.actListToActIdList(actSellList.getBase())
        return actSellList,  aidList

    def ifTradedRecently(self, sym,  bs=None):
        hobj = self._getRefObjForSymbol(sym, bs)
        if hobj:
            if hobj.canBeSold():
                return False,  None
        else:
            return False, None
        return True,  hobj
    def boughtRecentlyForAct(self, sym, act, no_of_days=30):
        hobj = self.getLastBoughtForAcctAndSym(sym=sym, acct=act)
        if hobj is None:
            return False
        return True

    def boughtRecently(self, sym, no_of_days=30):
        accts = self.getHoldingAccounts()
        actBuyList = BaseList()
        for act in accts.getBase():
            if self.boughtRecentlyForAct(sym, no_of_days=no_of_days,  act=act):
                actBuyList.append(act)
        return actBuyList

    def canSell(self, sym, no_of_days=30):
        if self.boughtRecently(sym, no_of_days=no_of_days):
            return False
        return True

    def idle_trade_qry(self, no_of_days=90):
        df = self.getDF()
        DateCol = 'datepd'
        df[DateCol] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

        # Define the date range (last six months from today)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=no_of_days)

        recent_traded = df[(df[DateCol] >= start_date) & (df[DateCol] <= end_date)]
        active_symbols = recent_traded[Symbol].unique()
        self.idle_symbols = list(set(self.all_symbols) - set(active_symbols))
        return self.idle_symbols

    def identyfyIdleSecurities(self, noactivity_in_days=180):
        symbols_by_date = self.getSymbolsWithLastTradedDate()
        today = datetime.date.today()
        # days_idle = (today - noactivity_in_days).dt.days
        self.idle_symbols = BaseSet()
        return

    def history_since(self, sdate="09/15/2024"):
        self.buy_bucket = BaseSet()
        self.sell_bucket = BaseSet()
        for item in self.getBase():
            if isinstance(item, History):
                if (item.Date > sdate):
                    if item.isBuy():
                        self.buy_bucket.append(item.Symbol.getBase())
                    else:
                        self.sell_bucket.append(item.Symbol.getBase())
        return

    def list_approved_since(self):
        # return
        self.sell_bucket.print()
        self.buy_bucket.print()
        return

    def isApproved2Buy(self, sym):
        return self.buy_bucket._exists(sym)
    def isApproved2Sell(self, sym):
        return self.sell_bucket._exists(sym)

    def prep_summ_df(self):
        # Sample Data (without 'trade_type', using quantity sign to differentiate buy/sell)
        # data = {
        #     'trade_id': [1, 2, 3, 4, 5],
        #     'symbol': ['AAPL', 'GOOG', 'AAPL', 'GOOG', 'AMZN'],
        #     'price': [150.0, 2800.0, 155.0, 2850.0, 3500.0],
        #     'quantity': [10, -5, 15, -8, 2],  # Negative quantity for sell trades
        #     'trade_date': ['2023-05-10', '2023-05-11', '2023-05-12', '2023-05-13', '2023-05-14']
        # }

        # return
        history_trades_df = self.getDF()
        # history_trades_df[Price].replace({'\,': ''})
        history_trades_df['price'] = history_trades_df[Price].replace({'\$': ''}, regex=True).replace({',': ''}, regex=True).astype(float)
        # Convert 'trade_date' to datetime for any date-based operations
        history_trades_df['trade_date'] = pd.to_datetime(history_trades_df[Date])
        trade_count = len(history_trades_df)

        # Group by 'symbol' and aggregate data
        self.summary_df = history_trades_df.groupby(Symbol).agg(
            qty_buy=(Quantity, lambda x: x[x > 0].sum()),  # Sum of positive quantities for buy trades
            qty_sell=(Quantity, lambda x: x[x < 0].sum()),  # Sum of negative quantities for sell trades
            avg_price_buy=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].mean()),
            # Average buy price
            avg_price_sell=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].mean()),
            first_trade_price=('price', 'last'),  # Last trade price
            last_trade_price=('price', 'first'),
            last_trade_qty=(Quantity, 'first'),
            # total_buy_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].sum()),
            # # Total buy amount
            # total_sell_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].sum()),
            # Average sell price
            # trade_count=('trade_id', 'count'),  # Count of trades
            first_trade_date=('trade_date', 'min'),  # First trade date
            last_trade_date=('trade_date', 'max')  # Last trade date
        ).reset_index()

        # Calculate the gain/loss (considering negative quantity for sell)
        self.summary_df['gain_loss'] = (self.summary_df['avg_price_sell'] - self.summary_df['avg_price_buy']) * abs(
            self.summary_df['qty_sell'])
        return

    def getSymbolsWithLastTradedDate(self):
        self.prep_summ_df()
        self.summary_df['last_trade_date'] = pd.to_datetime(self.summary_df['last_trade_date'])
        self.summary_df = self.summary_df.sort_values(by='last_trade_date', ascending=False)
        return self.summary_df

ModuleNotFoundError: No module named 'core'